In [1]:
import os
import argparse
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from utils import preprocess_data, SequenceDataset, train_model, score_model, log, get_predictions, plot_predictions
from model import LSTMRegression

/Users/twitter/anaconda3/envs/py310_ts/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/nyc_subway_turnstile_sample.csv')
df.shape

(26208, 17)

In [3]:
df

,time,14 ST-UNION SQ_entries,WORLD TRADE CTR_entries,TIMES SQ-42 ST_entries,SPRING ST_entries,GRD CNTRL-42 ST_entries,DEKALB AV_entries,7 AV_entries,GREENPOINT AV_entries,14 ST-UNION SQ_exits,WORLD TRADE CTR_exits,TIMES SQ-42 ST_exits,SPRING ST_exits,GRD CNTRL-42 ST_exits,DEKALB AV_exits,7 AV_exits,GREENPOINT AV_exits
0,2016-12-31 00:00:00,1192.0,28.0,434.0,55.0,126.0,178.0,230.0,35.0,369.0,42.0,355.0,46.0,294.0,78.0,227.0,34.0
1,2016-12-31 01:00:00,1192.0,28.0,434.0,55.0,126.0,178.0,230.0,35.0,369.0,42.0,355.0,46.0,294.0,78.0,227.0,34.0
2,2016-12-31 02:00:00,1192.0,28.0,434.0,55.0,126.0,178.0,230.0,35.0,369.0,42.0,355.0,46.0,294.0,78.0,227.0,34.0
3,2016-12-31 03:00:00,1192.0,28.0,434.0,55.0,126.0,178.0,230.0,35.0,369.0,42.0,355.0,46.0,294.0,78.0,227.0,34.0
4,2016-12-31 04:00:00,1192.0,28.0,434.0,55.0,126.0,178.0,230.0,35.0,369.0,42.0,355.0,46.0,294.0,78.0,227.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26203,2019-12-27 19:00:00,6656.0,2186.0,5738.0,1151.0,4822.0,1756.0,2177.0,255.0,5101.0,1073.0,3675.0,196.0,2059.0,661.0,1865.0,552.0
26204,2019-12-27 20:00:00,6656.0,677.0,5738.0,538.0,4822.0,724.0,2177.0,255.0,5101.0,483.0,3675.0,57.0,2059.0,401.0,1865.0,299.0
26205,2019-12-27 21:00:00,6656.0,677.0,5738.0,538.0,4822.0,724.0,2177.0,255.0,5101.0,483.0,3675.0,57.0,2059.0,401.0,1865.0,299.0
26206,2019-12-27 22:00:00,6656.0,677.0,5738.0,538.0,4822.0,724.0,2177.0,255.0,5101.0,483.0,3675.0,57.0,2059.0,401.0,1865.0,299.0


In [9]:
df1 = pd.read_parquet('../data/mta_subway_221231_100wk_dbscan.parquet')
df1.shape

(16800, 758)

In [10]:
df1

,1 AV_ent,1 AV_ex,103 ST-CORONA_ent,103 ST-CORONA_ex,103 ST_ent,103 ST_ex,104 ST_ent,104 ST_ex,110 ST_ent,110 ST_ex,...,WOODLAWN_ent,WOODLAWN_ex,WORLD TRADE CTR_ent,WORLD TRADE CTR_ex,WTC-CORTLANDT_ent,WTC-CORTLANDT_ex,YORK ST_ent,YORK ST_ex,ZEREGA AV_ent,ZEREGA AV_ex
TIME,,,,,,,,,,,,,,,,,,,,,
2021-01-23 00:00:00,12.00,63.50,233.25,37.50,12.00,31.50,31.50,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.00,5.75,0.00,0.0,0.25,6.0
2021-01-23 01:00:00,12.00,63.50,233.25,37.50,12.00,31.50,31.50,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.00,5.75,0.00,0.0,0.25,6.0
2021-01-23 02:00:00,12.00,63.50,233.25,37.50,12.00,31.50,31.50,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.00,5.75,0.00,0.0,0.25,6.0
2021-01-23 03:00:00,12.00,63.50,233.25,37.50,12.00,31.50,31.50,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.00,5.75,0.00,0.0,0.25,6.0
2021-01-23 04:00:00,12.00,63.50,233.25,37.50,12.00,31.50,31.50,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.00,5.75,0.00,0.0,0.25,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 19:00:00,520.50,974.75,559.25,1689.75,603.75,952.75,74.50,151.50,267.50,518.25,...,120.0,85.00,467.00,501.5,357.75,360.25,221.25,184.5,37.25,122.5
2022-12-23 20:00:00,205.75,498.25,164.75,887.00,603.75,952.75,20.75,78.25,83.00,246.00,...,50.5,49.50,110.00,232.5,357.75,360.25,221.25,184.5,37.25,122.5
2022-12-23 21:00:00,205.75,498.25,164.75,887.00,603.75,952.75,20.75,78.25,83.00,246.00,...,50.5,49.50,110.00,232.5,357.75,360.25,221.25,184.5,37.25,122.5


In [11]:
df1.index

DatetimeIndex(['2021-01-23 00:00:00', '2021-01-23 01:00:00',
               '2021-01-23 02:00:00', '2021-01-23 03:00:00',
               '2021-01-23 04:00:00', '2021-01-23 05:00:00',
               '2021-01-23 06:00:00', '2021-01-23 07:00:00',
               '2021-01-23 08:00:00', '2021-01-23 09:00:00',
               ...
               '2022-12-23 14:00:00', '2022-12-23 15:00:00',
               '2022-12-23 16:00:00', '2022-12-23 17:00:00',
               '2022-12-23 18:00:00', '2022-12-23 19:00:00',
               '2022-12-23 20:00:00', '2022-12-23 21:00:00',
               '2022-12-23 22:00:00', '2022-12-23 23:00:00'],
              dtype='datetime64[ns]', name='TIME', length=16800, freq=None)